In [2]:
import pandas as pd
import numpy as np
import sqlite3
import importlib
from datetime import datetime

# keeping company information in additional file
import data_file

In [ ]:
importlib.reload(data_file)

In [4]:
sales_person_n = data_file.sales_person_n
exlbu=data_file.exlbu
exlpanrter=data_file.exlpanrter

# Get current year and month
now = datetime.now()
last_month = now
# Format last month as string in YYYYMM format
year_month = last_month.strftime("%Y%m")

In [5]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"
df_orders = pd.read_sql_query(query, conn)
conn.close()

conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [6]:
# for testing purposes I keep this lines
wdf = df_customers.copy()
dfc = df_orders.copy()

In [7]:
wdf.loc[wdf['tier'].isna(), 'tier'] = 'Direct'
wdf.loc[wdf['tier'] == 'None', 'tier'] = 'Direct'
wdf.loc[wdf['tier_new'].isna(), 'tier_new'] = 'Direct'
wdf.loc[wdf['tier_new'] == 'None', 'tier_new'] = 'Direct'
wdf.loc[wdf[exlpanrter] == 'None', exlpanrter] = 'NA'
wdf.loc[wdf[exlpanrter].isna(), exlpanrter] = 'NA'

In [8]:
bu_defin = pd.read_excel('data_files/bu_defin.xlsx')
bu_defin = bu_defin.loc[:, 'bu2':]
bu_defin['for_bu'] = bu_defin['for_bu'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].astype(str)
bu_defin['bu2'] = bu_defin['bu2'].str.strip()

In [9]:
order_data = dfc.loc[:,['company_code_n', 'year_month', 'FY', 
       'bu2', 'sales_person_n', 'sales_order_so', 'sold_to_customer',
       'sold_to_customer_n','order_intake_amount_eur', 'ec_eu_customer_n']]

order_data['bu2'] = order_data['bu2'].astype(str)
order_data['bu2'] = order_data['bu2'].str.strip()

# add bu column for future reports
order_data = order_data.merge(bu_defin, how='left' )

In [10]:
print('should be 0 =', len(order_data) - len(dfc))
print('should be 0 =', order_data.order_intake_amount_eur.sum() - dfc.order_intake_amount_eur.sum())

should be 0 = 0
should be 0 = 0.0


In [11]:
order_columns = list(order_data.columns) + ['customer_name', 'indirect_direct', 'tier', 'tier_new', 'type', 'countries']

In [12]:
# special case with specific SO for agent
agent_so = data_file.special_so
agent_so_add = data_file.special_so_add 
agent_so_two = data_file.special_so_two 
agent_one = data_file.agent_one


special_so_three = data_file.special_so_three
agent_three = data_file.agent_three

so_order_data = order_data[(order_data['sales_order_so'] == agent_so) |(order_data['sales_order_so'] == agent_so_add) | (order_data['sales_order_so'] == agent_so_two) | (order_data['sales_order_so'] == special_so_three)]

# exlude these so_order_data raws from df
order_data = order_data[~(
    (order_data['sales_order_so'] == agent_so) |
    (order_data['sales_order_so'] == agent_so_add) |
    (order_data['sales_order_so'] == agent_so_two) |
    (order_data['sales_order_so'] == special_so_three)
)]

so_order_data.loc[so_order_data['sales_order_so'] != special_so_three, 'customer_name'] = agent_one
so_order_data.loc[so_order_data['sales_order_so'] == special_so_three, 'customer_name'] = agent_three

so_order_data.loc[:, 'indirect_direct'] = 'Indirect'
so_order_data.loc[:, 'tier'] = 'Channel Partner'
so_order_data.loc[:, 'tier_new'] = 'Channel Partner'
so_order_data.loc[:, 'type'] = 'Agent'

# Create a dictionary mapping sold_to_customer values to corresponding countries
coutr = df_customers.copy()
coutr['sold_to_customer'] = coutr['sold_to_customer'].astype(str)
customer_country_map = coutr.set_index('sold_to_customer')['countries'].to_dict()

# Fill missing values in so_order_data['countries'] using the mapping dictionary
so_order_data['countries'] = so_order_data['sold_to_customer'].map(customer_country_map)

In [13]:
# special case with specific cutomers for agent
partners_data = wdf[(wdf['tier'] != 'Direct') | (wdf[exlpanrter] != 'NA')]
agent_two = data_file.agent_two
partners_data.loc[partners_data[exlpanrter] != 'NA', 'customer_name'] = agent_two
partners_data = partners_data.loc[:, ['sold_to_customer', 'agent_person', 'company_code_n', 'sold_to_customer_n','customer_name', 
       'indirect_direct', 'channel', 'type', 'tier', 'tier_new', 'countries']]

partners_data['sold_to_customer'] = partners_data['sold_to_customer'].astype(str)

In [14]:
agent_partners = partners_data[partners_data['agent_person'].notna()]
agent_partners = agent_partners[agent_partners['agent_person'] != 'None']
# agent sales records preparation
agents_order_data = order_data[order_data['sales_person_n'].isin(sales_person_n)]
agents_order_data = agents_order_data.merge(agent_partners, left_on='sales_person_n', right_on='sold_to_customer_n', how='left')
agents_order_data = agents_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
agents_order_data.columns = agents_order_data.columns.str.replace('_x', '')
agents_order_data = agents_order_data.loc[:, order_columns]
agents_order_data['type'] = 'Agent'

# Fill missing values in so_order_data['countries'] using the mapping dictionary
agents_order_data['countries'] = agents_order_data['sold_to_customer'].map(customer_country_map)

In [15]:
# exclude agents sales from order data not to double lines
order_data = order_data[~order_data['sales_person_n'].isin(sales_person_n)]
# all partners except agents data preparation
other_order_data = order_data[order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])]
other_order_data = other_order_data.merge(partners_data, left_on='sold_to_customer', right_on='sold_to_customer', how='left')
other_order_data = other_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
other_order_data.columns = other_order_data.columns.str.replace('_x', '')
other_order_data = other_order_data.loc[:, order_columns]
# Update values in columns based on the specific value
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'tier'] = 'Channel Partner'
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'tier_new'] = 'Channel Partner'
other_order_data.loc[other_order_data['indirect_direct'] == 'Direct', 'type'] = 'Agent'

In [16]:
direct_order_data = order_data[~order_data['sold_to_customer'].isin(partners_data['sold_to_customer'])]
wdf['sold_to_customer'] = wdf['sold_to_customer'].astype(str)
direct_order_data = direct_order_data.merge(wdf, left_on='sold_to_customer', right_on='sold_to_customer', how='left')
direct_order_data = direct_order_data.filter(regex='^(?!.*_y)')
# Exclude '_x' from all column names
direct_order_data.columns = direct_order_data.columns.str.replace('_x', '')
direct_order_data = direct_order_data.loc[:, order_columns]

In [17]:
print('should be = 0: ', len(other_order_data) + len(direct_order_data) + len(agents_order_data) + len(so_order_data) - len(dfc))
print('should ~ 0 :', round((other_order_data.order_intake_amount_eur.sum() + direct_order_data.order_intake_amount_eur.sum() + agents_order_data.order_intake_amount_eur.sum() + so_order_data.order_intake_amount_eur.sum() - dfc.order_intake_amount_eur.sum()), 2))

should be = 0:  0
should ~ 0 : -0.0


In [18]:
# combining all the sales records
full_data = pd.concat([direct_order_data, other_order_data, agents_order_data, so_order_data])
full_data.reset_index(inplace=True, drop=True)
#full_data.drop_duplicates(inplace=True)

In [19]:
# for testing purposes I keep this lines
full_data_df = full_data.copy()
print('should ~ 0 :', round((full_data.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [20]:
#clean other division products

# define a list of products which I want to mark separetely
product_list = data_file.tm_product_list
tm_threshold = 0.9

# define total sum of orders for each customer
total_sum = full_data_df.groupby('customer_name')['order_intake_amount_eur'].sum()
total_sum = total_sum.rename('total_OI')
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, total_sum, on='customer_name', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['total_OI'] = full_data_df['total_OI'].fillna(0)

# Calculate the sum of sales for the products which I want to mark across all companies
total_tm = full_data_df[full_data_df['bu2'].isin(product_list)].groupby('customer_name')['order_intake_amount_eur'].sum().reset_index()
# Rename the 'order_intake_amount_eur' column to 'tm_sum'
total_tm.rename(columns={'order_intake_amount_eur': 'tm_sum'}, inplace=True)
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, total_tm, on='customer_name', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['tm_sum'] = full_data_df['tm_sum'].fillna(0)

# calculate the percentage of sales for the products in the list for each company
full_data_df['tm_share'] = full_data_df['tm_sum'] / full_data_df['total_OI']
full_data_df['tm_share'] = full_data_df['tm_share'].fillna(0)

# label companies where the percentage of sales for all products in the list is greater than treshold value
full_data_df['tm_check'] = 'no'
full_data_df.loc[full_data_df['tm_share'] > tm_threshold, 'tm_check'] = 'yes'

In [21]:
print('should ~ 0 :', round((full_data_df.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [22]:
# marking special case

special_one = data_file.special_one
special_one_office = data_file.special_one_office
# Filter the DataFrame based on the conditions
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] == special_one_office) , 'tm_check'] = 'no'
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] != special_one_office) , 'tier'] = 'Independent Sales Company'
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] != special_one_office) , 'tier_new'] = 'Independent Sales Company'
full_data_df.loc[(full_data_df['customer_name'] == special_one) & (full_data_df['company_code_n'] != special_one_office) , 'type'] = 'Trading Firm'

special_two = data_file.special_two
special_two_file_name = data_file.special_two_file_name

# Filter the DataFrame based on the conditions
# Convert 'data' column to datetime format
full_data_df['FY'] = pd.to_datetime(full_data_df['FY'])

special_two_so = pd.read_excel(special_two_file_name)
condition_a = full_data_df['sales_person_n'] != special_two
condition_b = full_data_df['sales_person_n'] == special_two
condition_c = full_data_df['sales_order_so'].isin(special_two_so['sales_order_so'])
condition_d = full_data_df['FY'].dt.year > 2022

full_data_df.loc[(condition_b & condition_c)| (condition_b & condition_d) , 'customer_name'] = special_two.upper()
full_data_df.loc[(condition_b & condition_c)| (condition_b & condition_d) , 'tier_new'] = 'Channel Partner'
full_data_df.loc[(condition_b & condition_c)| (condition_b & condition_d) , 'tier'] = 'Channel Partner'
full_data_df.loc[(condition_b & condition_c)| (condition_b & condition_d) , 'type'] = 'Agent'

In [23]:
print('should ~ 0 :', round((full_data_df.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [24]:
tm_exclusion_office = data_file.tm_exclusion_office
special_three = data_file.special_three
special_four = data_file.special_four
special_five = data_file.special_five
special_six = data_file.special_six
special_six_office = data_file.special_six_office
special_seven = data_file.special_seven


# eclusion case for t&m 
exl_condition_a = (full_data_df['FY'].dt.year == 2022) | (full_data_df['FY'].dt.year == 2021)
exl_condition_at = full_data_df['FY'].dt.year < 2023
exl_condition_b = full_data_df['company_code_n'] != tm_exclusion_office
exl_condition_c = full_data_df['tm_check'] == 'yes'
exl_condition_d = full_data_df['sold_to_customer'] == special_three
exl_condition_e = full_data_df['customer_name'] == special_four
exl_condition_f = full_data_df['customer_name'] == special_five

exl_condition_j = full_data_df['customer_name'] == special_six
exl_condition_i = full_data_df['company_code_n'].isin(special_six_office)

exl_condition_k = full_data_df['customer_name'].isin(special_seven)

#full_data_df.loc[exl_condition_at & exl_condition_k, 'tm_check'] = 'new'
#full_data_df.loc[exl_condition_at & exl_condition_d, 'tm_check'] = 'new'
#full_data_df.loc[exl_condition_j & exl_condition_i & exl_condition_at, 'tm_check'] = 'new'

full_data_df.loc[ (exl_condition_a & exl_condition_e) | (exl_condition_a & exl_condition_f) | (exl_condition_a & exl_condition_b & exl_condition_c), 'tm_check'] = 'no'

In [25]:
print('should ~ 0 :', round((full_data_df.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [26]:
special_eight = data_file.special_eight[0]
exl_condition_special_eight_name = full_data_df['customer_name'] == special_eight
exl_condition_special_eight = full_data_df['FY'].dt.year > 2022

full_data_df.loc[ exl_condition_special_eight_name & exl_condition_special_eight , 'tier_new'] = 'Indirect Business Partner'
print('should ~ 0 :', round((full_data_df.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [27]:
# Define the desired column order
column_order = ['sold_to_customer','customer_name', 'sold_to_customer_n', 'ec_eu_customer_n' ,'company_code_n', 'countries', 'indirect_direct', 'tier', 'tier_new', 'type', 'year_month', 'FY', 
       'bu2', 'for_bu', 'sales_order_so', 'sales_person_n','order_intake_amount_eur', 'tm_check', 'tm_share']

# Reorder the columns
full_data_df = full_data_df[column_order]

In [28]:
# other way to indentify T&M customers

#clean other division products

# define a list of products which I want to mark separetely
product_list = data_file.tm_product_list
pers_tm_threshold = 0.5

# define total sum of orders for each customer
person_total_sum = full_data_df.groupby('sales_person_n')['order_intake_amount_eur'].sum()
person_total_sum = person_total_sum.rename('person_total_OI')
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, person_total_sum, on='sales_person_n', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['person_total_OI'] = full_data_df['person_total_OI'].fillna(0)

# Calculate the sum of sales for the products which I want to mark across all companies
person_total_tm = full_data_df[full_data_df['bu2'].isin(product_list)].groupby('sales_person_n')['order_intake_amount_eur'].sum().reset_index()
# Rename the 'order_intake_amount_eur' column to 'tm_sum'
person_total_tm.rename(columns={'order_intake_amount_eur': 'pers_tm_sum'}, inplace=True)
# Merge with the original DataFrame to include all products and companies
full_data_df = pd.merge(full_data_df, person_total_tm, on='sales_person_n', how='left')
# Fill NaN values in 'sum' column with '0'
full_data_df['pers_tm_sum'] = full_data_df['pers_tm_sum'].fillna(0)

# calculate the percentage of sales for the products in the list for each company
full_data_df['pers_tm_share'] = full_data_df['pers_tm_sum'] / full_data_df['person_total_OI']
full_data_df['pers_tm_share'] = full_data_df['pers_tm_share'].fillna(0)

# label companies where the percentage of sales for all products in the list is greater than treshold value
full_data_df['pers_tm_check'] = 'no'
full_data_df.loc[full_data_df['pers_tm_share'] > pers_tm_threshold, 'pers_tm_check'] = 'yes'

print('should ~ 0 :', round((full_data_df.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [29]:
# Filter companies with only 'yes' values in column P
filtered_companies = full_data_df[full_data_df['pers_tm_check'] == 'yes']['sold_to_customer'].tolist()
filtered_companies_no = full_data_df[full_data_df['pers_tm_check'] == 'no']['sold_to_customer'].tolist()
filtered_companies = [x for x in filtered_companies if x not in filtered_companies_no]
filtered_companies = list(set(filtered_companies))

full_data_df['pure tm'] = 'no'
full_data_df.loc[full_data_df['sold_to_customer'].isin(filtered_companies), 'pure tm'] = 'yes'

print('should ~ 0 :', round((full_data_df.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [30]:
special_nine = data_file.special_nine
add_condition_a = full_data_df['sold_to_customer'] == special_nine

full_data_df.loc[add_condition_a, 'tm_check'] = 'no'
full_data_df.loc[add_condition_a, 'pure tm'] = 'no'

In [31]:
condition_one = full_data_df['tier'] == 'Independent Sales Company'
condition_two = full_data_df['FY'].dt.year == 2022

full_data_df.loc[condition_one & condition_two, 'pure tm'] = 'no'

print('should ~ 0 :', round((full_data_df.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(full_data_df) - len(df_orders))

should ~ 0 : -0.0
should = 0 : 0


In [32]:
# additional country data addition

rs_df = pd.read_excel('data_files/rs_data.xlsx')
# Convert the date column to datetime type
rs_df['year_month'] = pd.to_datetime(rs_df['year_month'])
rs_df['m_num'] = rs_df['year_month'].dt.strftime('%m')
# Create a new column 'month_year' with the desired format "202003"
rs_df['year_month'] = rs_df['year_month'].dt.strftime('%Y%m')
rs_df['m_num'] = rs_df['m_num'].astype(int)

def apply_shift(value):
    if value >= 4:
        return value - 3
    else:
        return value + 9
    
# Apply the shift to 'm_num 4'
rs_df['m_num'] = rs_df['m_num'].apply(apply_shift)


temp = full_data_df.copy()

temp['year_month'] = temp['year_month'].astype(str)
temp['m_num'] = temp['year_month'].str[-2:].astype(int)
temp['m_num'] = temp['m_num'].apply(apply_shift)

temp = pd.concat([temp, rs_df])

temp = temp[['sold_to_customer', 'customer_name', 'sold_to_customer_n',
       'ec_eu_customer_n', 'company_code_n', 'countries', 'indirect_direct',
       'tier_new', 'type', 'year_month', 'm_num', 'FY', 'bu2', 'for_bu',
       'sales_order_so', 'sales_person_n', 'order_intake_amount_eur',
       'tm_check', 'pure tm']]

In [33]:
print('should ~ 0 :', round((temp.order_intake_amount_eur.sum() - df_orders.order_intake_amount_eur.sum() - rs_df.order_intake_amount_eur.sum()), 2))
print('should = 0 :', len(temp) - len(df_orders) - len(rs_df))

should ~ 0 : -0.0
should = 0 : 0


In [34]:
full_oi_data_df_final = temp.copy()

In [78]:
# for testing
temp = full_oi_data_df_final.copy()

In [35]:
# add budget information
targets = pd.read_excel('data_files/outcome/target.xlsx')
targets['order_intake_amount_eur'] = 0
# Convert the 'FY' column to datetime format
targets['FY'] = pd.to_datetime(targets['FY'], format='%d-%m-%Y')
temp['target_order_intake_amount_eur'] = 0
temp = pd.concat([temp, targets], ignore_index=True)

In [36]:
bu_list = ['Flowmeters', 'Services', 'Others', 'Netsol', 'Analytics',
       'Transmitters', 'Systems&ADS', 'T&M']

In [37]:
opco_list = ['YEF-A', 'YEF-B', 'YEF-CH', 'YEF-CZ', 'YEF-E', 'YEF-F',
       'YEF-G', 'YEF-GB', 'YEF-HU', 'YEF-I', 'YEF-NL', 'YEF-PL',
       'YEF-RO', 'YEF-SK', 'YEF-TR', 'YEF-RS', 'YEF-BG', 'RYG']

In [38]:
#temp = temp[temp['company_code_n'].isin(opco_list)]

In [39]:
#result_chp_opcos = {}
result_isc_opcos = {}
result_ibp_opcos = {}

def getresult(df, opco):
    
    # addition of target numbers for two business tiers
    #condition_chp = df['tier_new'] == 'Channel Partner'
    condition_isc = df['tier_new'] == 'Independent Sales Company'
    condition_ibp = df['tier_new'] == 'Indirect Business Partner'
    condition_tm = (df['tm_check'] == 'no') & (df['pure tm'] == 'no')
    
    #opco_result_chp = df[ (df['company_code_n'] == opco) & (df['FY'].dt.year == 2022) & condition_chp & condition_tm]['order_intake_amount_eur'].sum()
    #opco_target_chp = df[ (df['company_code_n'] == opco) & (df['FY'].dt.year == 2023) & condition_chp & condition_tm]['target_order_intake_amount_eur'].sum()
    
    #opco_target_chp_should_be_by_month = (round(opco_result_chp * 1.1, -3) - opco_target_chp) / 12
    #opco_target_chp_should_be_by_month = round(opco_target_chp_should_be_by_month, -3)
    
    opco_result_isc = df[ (df['company_code_n'] == opco) & (df['FY'].dt.year == 2022) & condition_isc & condition_tm]['order_intake_amount_eur'].sum()
    opco_target_isc = df[ (df['company_code_n'] == opco) & (df['FY'].dt.year == 2023) & condition_isc & condition_tm]['target_order_intake_amount_eur'].sum()
    
    opco_target_isc_should_be_by_month = (round(opco_result_isc * 1.1, -3) - opco_target_isc) / 12
    opco_target_isc_should_be_by_month = round(opco_target_isc_should_be_by_month, -3)
    
    opco_result_ibp = df[(df['company_code_n'] == opco) & (df['FY'].dt.year == 2022) & condition_ibp & condition_tm]['order_intake_amount_eur'].sum()
    opco_target_ibp = df[(df['company_code_n'] == opco) & (df['FY'].dt.year == 2023) & condition_ibp & condition_tm]['target_order_intake_amount_eur'].sum()

    opco_target_ibp_should_be_by_month = (round(opco_result_ibp * 1.1, -3) - opco_target_ibp) / 12
    opco_target_ibp_should_be_by_month = round(opco_target_ibp_should_be_by_month, -3)
    
    return opco_target_isc_should_be_by_month, opco_target_ibp_should_be_by_month  # opco_target_chp_should_be_by_month, 


for opco in opco_list:
    #result_chp_opcos[opco] = getresult(temp, opco)[0]
    result_isc_opcos[opco] = getresult(temp, opco)[0]
    result_ibp_opcos[opco] = getresult(temp, opco)[1]

In [40]:
fy_23 =  [202304, 202305, 202306, 202307, 202308, 202309, 202310, 202311, 202312, 202401, 202402, 202403]
month_num =  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

"""
list_of_df_chp = []

for key, value in result_chp_opcos.items():
    chp_target_df = pd.DataFrame(columns=temp.columns)
    chp_target_df.loc[0, 'company_code_n'] = key
    chp_target_df.loc[0, 'tier_new'] = 'Channel Partner'
    chp_target_df.loc[0, 'tier'] = 'Channel Partner'
    chp_target_df.loc[0, 'pure tm'] = 'no'
    chp_target_df.loc[0, 'tm_check'] = 'no'
    if value >= 0:
        chp_target_df.loc[0, 'target_order_intake_amount_eur'] = value
    else:
        chp_target_df.loc[0, 'target_order_intake_amount_eur'] = 0
    chp_target_df.loc[0, 'order_intake_amount_eur'] = 0
    chp_target_df = pd.concat([chp_target_df] * 12, ignore_index=True)
    chp_target_df['year_month'] = fy_23
    chp_target_df['m_num'] = month_num
    chp_target_df['FY'] = temp['FY'][temp['FY'].dt.year == 2023].unique()
    list_of_df_chp.append(chp_target_df)

full_chp_target_df = pd.DataFrame(columns=temp.columns)


    
for idf in list_of_df_chp:
    full_chp_target_df = pd.concat([full_chp_target_df, idf], ignore_index=True)
"""

list_of_df_isc = []

for key, value in result_isc_opcos.items():
    isc_target_df = pd.DataFrame(columns=temp.columns)
    isc_target_df.loc[0, 'company_code_n'] = key
    isc_target_df.loc[0, 'tier_new'] = 'Independent Sales Company'
    isc_target_df.loc[0, 'tier'] = 'Independent Sales Company'
    isc_target_df.loc[0, 'pure tm'] = 'no'
    isc_target_df.loc[0, 'tm_check'] = 'no'
    isc_target_df.loc[0, 'target_order_intake_amount_eur'] = value
    isc_target_df.loc[0, 'order_intake_amount_eur'] = 0
    isc_target_df = pd.concat([isc_target_df] * 12, ignore_index=True)
    isc_target_df['year_month'] = fy_23
    isc_target_df['m_num'] = month_num
    isc_target_df['FY'] = temp['FY'][temp['FY'].dt.year == 2023].unique()
    list_of_df_isc.append(isc_target_df)

full_isc_target_df = pd.DataFrame(columns=temp.columns)
    
for idf in list_of_df_isc:
    full_isc_target_df = pd.concat([full_isc_target_df, idf], ignore_index=True)
    

list_df_ibp = []

for key, value in result_ibp_opcos.items():
    ibp_target_df = pd.DataFrame(columns=temp.columns)
    ibp_target_df.loc[0, 'company_code_n'] = key
    ibp_target_df.loc[0, 'tier_new'] = 'Indirect Business Partner'
    ibp_target_df.loc[0, 'tier'] = 'Indirect Business Partner'
    ibp_target_df.loc[0, 'pure tm'] = 'no'
    ibp_target_df.loc[0, 'tm_check'] = 'no'
    ibp_target_df.loc[0, 'target_order_intake_amount_eur'] = value
    ibp_target_df.loc[0, 'order_intake_amount_eur'] = 0
    ibp_target_df = pd.concat([ibp_target_df] * 12, ignore_index=True)
    ibp_target_df['year_month'] = fy_23
    ibp_target_df['m_num'] = month_num
    ibp_target_df['FY'] = temp['FY'][temp['FY'].dt.year == 2023].unique()
    list_df_ibp.append(ibp_target_df)

full_ibp_target_df = pd.DataFrame(columns=temp.columns)
    
for idf in list_df_ibp:
    full_ibp_target_df = pd.concat([full_ibp_target_df, idf], ignore_index=True)

In [41]:
#temp = pd.concat([temp, full_chp_target_df])
temp = pd.concat([temp, full_isc_target_df])
temp = pd.concat([temp, full_ibp_target_df])
temp = temp.reset_index(drop=True)

condition_tm = (temp['tm_check'] == 'no') & (temp['pure tm'] == 'no')


"""

temp = temp[condition_tm]

temp = temp[['sold_to_customer', 'customer_name', 'sold_to_customer_n',
       'ec_eu_customer_n', 'company_code_n', 'countries', 'indirect_direct', 
       'tier_new', 'type', 'year_month', 'FY', 'bu2', 'for_bu',
       'sales_order_so', 'sales_person_n', 'order_intake_amount_eur',
        'target_order_intake_amount_eur', 'm_num']]

"""
temp = temp[['sold_to_customer', 'customer_name', 'sold_to_customer_n',
       'ec_eu_customer_n', 'company_code_n', 'countries', 'indirect_direct', 
       'tier_new', 'type', 'year_month', 'FY', 'bu2', 'for_bu',
       'sales_order_so', 'sales_person_n', 'order_intake_amount_eur',
        'target_order_intake_amount_eur', 'm_num', 'tm_check','pure tm']]


# Create a pivot table from the DataFrame
temp_b = temp.copy()
temp_b['FY'] = temp_b['FY'].dt.year
pivot_table1 = temp_b.pivot_table(index='tier_new', columns='FY', values='order_intake_amount_eur', aggfunc='sum')
pivot_table2 = temp_b.pivot_table(index='tier_new', columns='FY', values='target_order_intake_amount_eur', aggfunc='sum')

# Create a dictionary mapping sold_to_customer values to corresponding countries
df_customers['sold_to_customer'] = df_customers['sold_to_customer'].astype(str)
customer_country_map = df_customers.set_index('sold_to_customer')['countries'].to_dict()

# Fill missing values in temp_df['countries'] using the mapping dictionary
temp['countries'] = temp['countries'].fillna(temp['sold_to_customer'].map(customer_country_map))

In [42]:
last_df = temp.copy()

last_df['sold_to_customer'] = last_df['sold_to_customer'].fillna('900000000')
last_df['customer_name'] = last_df['customer_name'].fillna('FB')

df = temp.copy()
# Step 1: Group by 'country_code_n' and find the most frequent country for each group
most_frequent_countries = df.groupby('company_code_n')['countries'].agg(lambda x: x.value_counts().idxmax())
# Step 2: Convert the pandas Series to a dictionary
country_code_to_country = most_frequent_countries.to_dict()
# Step 2: Fill empty values in the 'countries' column using the dictionary
last_df['countries'] = last_df['countries'].fillna(last_df['company_code_n'].map(country_code_to_country))

last_df['type'] = last_df['type'].fillna('FB')
last_df['bu2'] = last_df['bu2'].fillna('FB')
last_df['indirect_direct'] = last_df['indirect_direct'].fillna('Indirect')
last_df['for_bu'] = last_df['for_bu'].fillna('All')

In [39]:
writer = pd.ExcelWriter(f'data_files/outcome/results_{year_month}.xlsx')
# Save each DataFrame to a separate sheet in the same file
last_df.to_excel(writer, sheet_name='temp', index=False)
pivot_table1.to_excel(writer, sheet_name='pivot')
pivot_table2.to_excel(writer, sheet_name='pivot2')
# Save the file
writer.close()

In [40]:
# write previous pivot
writer = pd.ExcelWriter(f'data_files/outcome/last_pivot_{year_month}.xlsx')
pivot_table1.to_excel(writer, sheet_name='pivot')
writer.close()

In [ ]:
# understanding difference between last data and new
last_pivot = pd.read_excel(f'data_files/outcome/last_pivot.xlsx')

last_pivot = last_pivot.set_index('tier_new')
last_pivot= last_pivot.round(2)

pivot_table = pivot_table1.reset_index()
pivot_table = pivot_table.set_index('tier_new')
pivot_table = pivot_table.round(2)

pivot_table - last_pivot

In [43]:
temp = last_df.copy()

In [44]:
temp_so_checker = temp[['sales_order_so', 'sold_to_customer', 'customer_name', 'indirect_direct',
       'tier_new', 'type', 'FY', 'order_intake_amount_eur', 'company_code_n', 'countries']]

temp_so_checker = temp_so_checker[temp_so_checker['sold_to_customer'] != 999999999]
temp_so_checker['sum_column'] = temp_so_checker.groupby(['sales_order_so' , 'tier_new'])['order_intake_amount_eur'].transform('sum')
temp_so_checker = temp_so_checker.sort_values(by=['FY','sum_column'], ascending=[False, False])
temp_so_checker = temp_so_checker.reset_index(drop='True')
temp_so_checker = temp_so_checker.drop_duplicates(subset='sales_order_so', keep='first')

# Step 1: Count unique values in the column
value_counts = temp_so_checker['sales_order_so'].value_counts()

# Step 2: Filter values that occur more than once
values_gt_one = value_counts[value_counts > 1]

print(values_gt_one)

temp_so_checker = temp_so_checker[['sales_order_so', 'sold_to_customer', 'customer_name', 'indirect_direct', 'tier_new', 'type', 'company_code_n', 'countries']]

temp_so_checker = temp_so_checker.dropna(subset='sales_order_so')

Series([], Name: sales_order_so, dtype: int64)


In [45]:
# Group by 'sales_order_so' and select the first occurrence of each column within each group
mapped_data = temp.groupby('sales_order_so').agg({
    'sold_to_customer': 'first',
    'customer_name': 'first',
    'company_code_n': 'first',
    'countries': 'first',
    'indirect_direct': 'first',
    'tier_new': 'first',
    'type': 'first', 'tm_check': 'first', 'pure tm': 'first'
}).reset_index()

In [46]:
conn3 = sqlite3.connect('data_files/sales_database.db')
query = "SELECT * FROM sales"
df_sales = pd.read_sql_query(query, conn3)
conn3.close()

In [47]:
sales_mapped_df = df_sales.merge(mapped_data, on='sales_order_so', how='left', suffixes=('_X', ''))

In [48]:
sales_mapped = sales_mapped_df[~sales_mapped_df['sold_to_customer'].isna()]
sales_not_mapped = sales_mapped_df[sales_mapped_df['sold_to_customer'].isna()]

print(len(df_sales) - len(sales_mapped) - len(sales_not_mapped))

0


In [49]:
sales_not_mapped = sales_not_mapped[['fiscal_year', 'year_month', 'company_code_n_X', 'sales_order_so', 'bu',
       'bu_n', 'sales_person', 'sales_person_n', 'sold_to_customer_n_latest',
       'sold_to_customer_X', 'sold_to_customer_n', 'sold_to_country_n',
       'ec_eu_customer', 'ec_eu_customer_n', 'ec_eu_country_n',
       'ec_eu_industry_segment_n', 'ship_to_customer', 'ship_to_customer_n',
       'ship_to_country_n', 'ST', 'GP', 'Source']]


sales_not_mapped['pure tm'] = 'no'
sales_not_mapped.loc[sales_not_mapped['sold_to_customer_X'].isin(filtered_companies), 'pure tm'] = 'yes'
sales_not_mapped['tm_check'] = 'no'

sales_not_mapped = sales_not_mapped.dropna(subset='sold_to_customer_X')
sales_not_mapped = sales_not_mapped.rename(columns={'sold_to_customer_X': 'sold_to_customer'})
sales_not_mapped['sold_to_customer'] = sales_not_mapped['sold_to_customer'].astype(int)
sales_not_mapped['sold_to_customer'] = sales_not_mapped['sold_to_customer'].astype(str)

print(len(sales_not_mapped))

wdf_copy = wdf[['sold_to_customer', 'company_code_n',
        'customer_name', 
       'indirect_direct', 'channel', 'type', 'Horn', 
       'tier_new', 'countries']]

sales_not_mapped_merged = sales_not_mapped.merge(wdf_copy, on='sold_to_customer', how='inner', suffixes=('_X', ''))
sales_not_mapped_out = sales_not_mapped.merge(wdf_copy, on='sold_to_customer', how='outer', suffixes=('_X', ''), indicator=True)

print(len(sales_not_mapped_merged), len(sales_not_mapped_out))

23294
22917 35987


In [123]:
# write previous pivot
writer = pd.ExcelWriter(f'data_files/outcome/sales_not_mapped_out_{year_month}.xlsx')
sales_not_mapped_out.to_excel(writer, sheet_name='pivot')
writer.close()

In [94]:
# тут временно потеряю Horn? и те компании, которых нет в базе данных

In [50]:
sales_not_mapped = sales_not_mapped_merged.copy()

In [ ]:
sales_mapped['FY'] = pd.to_datetime(sales_mapped['fiscal_year'], format='%Y')   
#sales_mapped['m_num'] = sales_mapped['FY'].dt.month
sales_mapped['bu2'] = sales_mapped['bu'].astype(str)
sales_mapped['bu2'] = sales_mapped['bu2'].str.strip()
sales_mapped = sales_mapped.merge(bu_defin, how='left' )
sales_mapped = sales_mapped[['sold_to_customer', 'customer_name', 'sold_to_customer_n',
       'ec_eu_customer_n', 'company_code_n', 'countries', 'indirect_direct',
       'tier_new', 'type', 'year_month', 'FY', 'bu2', 'for_bu',
       'sales_order_so', 'sales_person_n', 'ST', 'GP', 'tm_check', 'pure tm']]


sales_not_mapped['FY'] = pd.to_datetime(sales_not_mapped['fiscal_year'], format='%Y')
#sales_not_mapped['m_num'] = sales_not_mapped['FY'].dt.month
sales_not_mapped['bu2'] = sales_not_mapped['bu'].astype(str)
sales_not_mapped['bu2'] = sales_not_mapped['bu2'].str.strip()
sales_not_mapped = sales_not_mapped.merge(bu_defin, how='left' )
sales_not_mapped = sales_not_mapped[['sold_to_customer', 'customer_name', 'sold_to_customer_n',
       'ec_eu_customer_n', 'company_code_n', 'countries', 'indirect_direct',
       'tier_new', 'type', 'year_month', 'FY', 'bu2', 'for_bu',
       'sales_order_so', 'sales_person_n', 'ST', 'GP', 'tm_check', 'pure tm']]

In [52]:
sales_mapped_df = pd.concat([sales_mapped, sales_not_mapped], ignore_index=True)

def apply_shift(value):
    if value >= 4:
        return value - 3
    else:
        return value + 9
    
sales_mapped_df['m_num'] = sales_mapped_df['year_month'].astype(str)
sales_mapped_df['m_num'] = sales_mapped_df['m_num'].str[-2:].astype(int)
sales_mapped_df['m_num'] = sales_mapped_df['m_num'].apply(apply_shift)

Making file for Tableau

In [53]:
condition_tm = (sales_mapped_df['tm_check'] == 'no') & (sales_mapped_df['pure tm'] == 'no')
sales_mapped_df = sales_mapped_df[condition_tm]

df = last_df.copy()
condition_tm = (df['tm_check'] == 'no') & (df['pure tm'] == 'no')
df = df[condition_tm]

In [54]:
#df = last_df.copy()

# Group by the relevant columns and aggregate the sum values
grouped_df = df.groupby(['company_code_n', 'sold_to_customer', 'customer_name', 'countries', 'tier_new', 'type', 'FY', 'year_month', 'm_num', 'bu2', 'for_bu']) \
                    .agg(order_intake_amount_eur=('order_intake_amount_eur', 'sum'),
                         target_order_intake_amount_eur=('target_order_intake_amount_eur', 'sum')) \
                    .reset_index()

# Create the new DataFrame with the specified columns
new_df = grouped_df[['company_code_n', 'sold_to_customer', 'customer_name', 'countries', 'tier_new', 'type', 'FY', 'year_month', 'm_num', 'bu2', 'for_bu', 'order_intake_amount_eur', 'target_order_intake_amount_eur']]

In [55]:
# Group by the relevant columns and aggregate the sum values
grouped_df_2 = sales_mapped_df.groupby(['company_code_n', 'sold_to_customer', 'customer_name', 'countries', 'tier_new', 'type', 'FY', 'year_month', 'm_num', 'bu2', 'for_bu']) \
                    .agg(sales_amount_eur=('ST', 'sum'),
                         gross_profit_amount_eur=('GP', 'sum')) \
                    .reset_index()

# Create the new DataFrame with the specified columns
new_df_2 = grouped_df_2[['company_code_n', 'sold_to_customer', 'customer_name', 'countries', 'tier_new', 'type', 'FY', 'year_month', 'm_num', 'bu2', 'for_bu', 'sales_amount_eur', 'gross_profit_amount_eur']]

In [56]:
full_results_dataframe = pd.concat([new_df, new_df_2], ignore_index=True)

In [57]:
# Columns to fill with NaN values
columns_to_fill = ['order_intake_amount_eur', 'target_order_intake_amount_eur', 'sales_amount_eur', 'gross_profit_amount_eur']

# Fill NaN values in the specified columns with 0
full_results_dataframe[columns_to_fill] = full_results_dataframe[columns_to_fill].fillna(0)

In [58]:
full_results_dataframe = full_results_dataframe.rename(columns={'sold_to_customer': 'ID', 'company_code_n':'OPCO', 'customer_name':'Partner Name', 'tier_new':'Tier','type':'Type', 'countries':'Country', 
                                                                'order_intake_amount_eur':'Order Intake',
                                                               'target_order_intake_amount_eur':'Target Order Intake',
                                                               'sales_amount_eur': 'Sales',
                                                               'gross_profit_amount_eur': 'Gross Profit'},
                                                      )

In [59]:
countries_to_exclude = ['New Caledonia', 'Brazil', 'Singapore', 'Utd.Arab Emir.', 'USA', 'China']
full_results_dataframe = full_results_dataframe[~full_results_dataframe['Country'].isin(countries_to_exclude)]

In [60]:
full_results_dataframe = full_results_dataframe[full_results_dataframe['OPCO'] != 'RYG']

full_results_dataframe.loc[full_results_dataframe['Country'] == 'Bosnia-Herz.', 'Country'] = 'Bosnia-Herzegovina'
full_results_dataframe.loc[full_results_dataframe['Country'] == 'Bosnia', 'Country'] = 'Bosnia-Herzegovina'

In [61]:
full_results_dataframe['year_month'] = full_results_dataframe['year_month'].astype(str)

In [62]:
full_results_dataframe['temp'] = pd.to_datetime(full_results_dataframe['year_month'], format='%Y%m')

# Shift the dates back by three months using the pandas DateOffset
full_results_dataframe['temp'] = full_results_dataframe['temp'] - pd.DateOffset(months=3)

full_results_dataframe['temp'] = pd.to_datetime(full_results_dataframe['temp'], format='%Y-%m-%d')
full_results_dataframe['FY'] = full_results_dataframe['temp']

full_results_dataframe.drop('temp', axis=1, inplace=True)

In [63]:
# Group the DataFrame by the specified columns and calculate the sum for the desired columns
grouped_df = full_results_dataframe.groupby(by=['OPCO', 'ID', 'Partner Name', 'Country', 'Tier', 'Type', 'FY', 'year_month', 'm_num', 'bu2', 'for_bu'])[['Order Intake',
                                                                    'Target Order Intake',
                                                                    'Sales',
                                                                    'Gross Profit']].sum().reset_index()

In [64]:
# Filter the DataFrame to keep only rows with 'Tier' = 'Direct'
direct_df = grouped_df[grouped_df['Tier'] == 'Direct']
indirect_df = grouped_df[~(grouped_df['Tier'] == 'Direct')]

# Group the 'direct_df' by the specified columns and calculate the sum for the desired columns
direct_grouped_df = direct_df.groupby(['OPCO', 'Country', 'FY', 'year_month', 'm_num', 'for_bu']).agg({
    'Order Intake': 'sum',
    'Target Order Intake': 'sum',
    'Sales': 'sum',
    'Gross Profit': 'sum',
    'Partner Name': lambda x: 'Direct_customers',  # Set 'Direct_name' for all 'Partner Name's
    'Type': lambda x: 'Direct_type',           # Set 'Direct_name' for all 'Type's
    'bu2': lambda x: 'combined',
    'Tier': lambda x: 'Direct'
}).reset_index()

In [65]:
tableau_df = pd.concat([direct_grouped_df, indirect_df], ignore_index=True)

In [66]:
tableau_df = tableau_df[['OPCO', 'Partner Name', 'Country', 'Tier', 'Type', 'FY',
       'year_month', 'm_num', 'bu2', 'for_bu', 'Order Intake',
       'Target Order Intake', 'Sales', 'Gross Profit']]

In [67]:
# Define a mapping dictionary for month abbreviations
month_map = {
    1: '1-Apr', 2: '2-May', 3: '3-Jun', 4: '4-Jul',
    5: '5-Aug', 6: '6-Sep', 7: '7-Oct', 8: '8-Nov',
    9: '9-Dec', 10: '10-Jan', 11: '11-Feb', 12: '12-Mar'
}

# Apply the mapping to the 'm_num' column
tableau_df['m_num'] = tableau_df['m_num'].map(month_map)

In [68]:
len(tableau_df)

75308

In [69]:
# Define the list of columns for which all values should be equal to 0
columns_to_check = ['Order Intake', 'Target Order Intake', 'Sales', 'Gross Profit']

# Use boolean indexing to filter out the rows where all the specified columns have 0 values
tableau_df = tableau_df[~(tableau_df[columns_to_check] == 0).all(axis=1)]

In [72]:
# there is some partners with double opcos, it is necessary to correct them
final_r = data_file.final_r
final_b = data_file.final_b
# change BG
tableau_df.loc[tableau_df['OPCO'] == final_b, 'OPCO'] = final_r

In [73]:
for_types_changes = tableau_df[tableau_df['Tier'] == 'Channel Partner']
# Group by 'Partner Name' and filter groups with count greater than 1 in 'OPCO'
result = for_types_changes.groupby('Partner Name').filter(lambda x: len(x['OPCO'].unique()) > 1)
# Get the unique companies from the result
unique_companies = result['Partner Name'].unique()
for_changing = for_types_changes[for_types_changes['Partner Name'].isin(unique_companies)]
for_changing = for_changing[for_changing['FY'].dt.year > 2021]

# Group by 'Partner Name' and 'OPCO', aggregate the sum of 'Order Intake'
opco_sum = for_changing.groupby(['Partner Name', 'OPCO'])['Order Intake'].sum().reset_index()

# Find the index of the maximum 'Order Intake' sum for each 'Partner Name'
idx = opco_sum.groupby('Partner Name')['Order Intake'].idxmax()

# Select the corresponding rows from 'opco_sum'
max_opco_per_partner = opco_sum.loc[idx]

# Create the new DataFrame with 'Partner Name' and the corresponding 'OPCO'
result_df = max_opco_per_partner[['Partner Name', 'OPCO']]

In [76]:
# Creating the mapping dictionary
mapping_dict_opcos = {}
for index, row in result_df.iterrows():
    mapping_dict_opcos[row['Partner Name']] = row['OPCO']

In [77]:
# Update 'OPCO' column based on conditions and mapping dictionary
for partner, opco in mapping_dict_opcos.items():
    tableau_df.loc[(tableau_df['Partner Name'] == partner) & (tableau_df['Tier'] == 'Channel Partner'), 'OPCO'] = opco
    
final_c_one = data_file.final_c_one
final_c = data_file.final_c
final_t_one = data_file.final_t_one
final_t = data_file.final_t

final_p_one = data_file.final_p_one
final_p = data_file.final_p

condition_c = tableau_df['Partner Name'] == final_c_one
condition_t = tableau_df['Partner Name'] == final_t_one
condition_p = tableau_df['Partner Name'] == final_p_one

tableau_df.loc[condition_c, 'OPCO'] = final_c
tableau_df.loc[condition_t, 'OPCO'] = final_t
tableau_df.loc[condition_p, 'OPCO'] = final_p

In [79]:
# write previous pivot
writer = pd.ExcelWriter(f'data_files/outcome/Indirect Business Data Set.xlsx')
tableau_df.to_excel(writer, sheet_name='full', index=False)
writer.close()

In [78]:
import os
import shutil

def copy_file_to_folders(source_file, target_folders):
    for folder in target_folders:
        destination = os.path.join(folder, os.path.basename(source_file))
        shutil.copy2(source_file, destination)
        print(f"File '{source_file}' copied to '{destination}'")

# Example usage
source_folder = 'data_files/outcome/'
source_file = os.path.join(source_folder, 'Indirect Business Data Set.xlsx')

print(source_file)

target_folders = [
    'Z:',
]


copy_file_to_folders(source_file, target_folders)

data_files/outcome/Indirect Business Data Set.xlsx
File 'data_files/outcome/Indirect Business Data Set.xlsx' copied to 'Z:Indirect Business Data Set.xlsx'
